# PyTorch Model + Transformer Example

This notebook demonstrates how to deploy a PyTorch model and a custom transformer. It uses cifar10 model model that accepts a tensor input. The transformer has preprocessing step that allows the user to send a raw image data and convert it to a tensor input.

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

In [ ]:
!pip install --upgrade -r requirements.txt > /dev/null

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 1. Initialize Merlin

### 1.1 Set Merlin Server

In [ ]:
import merlin

MERLIN_URL = "merlin-api-url"

merlin.set_url(MERLIN_URL)

### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`merlin.set_project(<project-name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the MLP UI.

In [ ]:
PROJECT_NAME = "sample"

merlin.set_project(PROJECT_NAME)

### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in Merlin is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by Merlin are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`merlin.set_model(<model_name>, <model_type>)` will set the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [ ]:
from merlin.model import ModelType

MODEL_NAME = "transformer-pytorch"

merlin.set_model(MODEL_NAME, ModelType.PYTORCH)

## 2. Train Model

### 2.1 Prepare Training Data

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

### 2.2 Create PyTorch Model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### 2.3 Train Model

In [ ]:
import torch.optim as optim

net = PyTorchModel()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

### 2.4 Check Prediction

In [ ]:
dataiter = iter(trainloader)
inputs, labels = dataiter.next()

predict_out = net(inputs[0:1])
predict_out

## 3. Deploy Model and Transformer

### 3.1 Serialize Model

In [ ]:
import os

model_dir = "pytorch-model"
model_path = os.path.join(model_dir, "model.pt")

torch.save(net.state_dict(), model_path)

### 3.2 Save PyTorchModel Class

We also need to save the PyTorchModel class and upload it to Merlin alongside the serialized model. The next cell will write the PyTorchModel we defined above to `pytorch-model/model.py` file.

In [ ]:
%%file pytorch-model/model.py
import torch.nn as nn
import torch.nn.functional as F

class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### 3.3 Create Model Version and Upload

`merlin.new_model_version()` is a convenient method to create a model version and start its development process. It is equal to following codes:

```
v = model.new_model_version()
v.start()
v.log_pytorch_model(model_dir=model_dir)
v.finish()
```

In [ ]:
# Create new version of the model
with merlin.new_model_version() as v:
    # Upload the serialized model to Merlin
    merlin.log_pytorch_model(model_dir=model_dir)

### 3.4 Deploy Model and Transformer

To deploy a model and its transformer, you must pass a `transformer` object to `deploy()` function. Each of deployed model version will have its own generated url.

In [ ]:
from merlin.resource_request import ResourceRequest
from merlin.transformer import Transformer

# Create a transformer object and its resources requests
resource_request = ResourceRequest(min_replica=1, max_replica=1, 
                                   cpu_request="100m", memory_request="200Mi")
transformer = Transformer(enabled=True,
                          image="gcr.io/kubeflow-ci/kfserving/image-transformer:latest",
                          resource_request=resource_request)

endpoint = merlin.deploy(v, transformer=transformer)

### 3.5 Send Test Request

In [ ]:
import json
import requests

with open(os.path.join("input-raw-image.json"), "r") as f:
    req = json.load(f)

resp = requests.post(endpoint.url, json=req)
resp.text

## 4. Clean Up

## 4.1 Delete Deployment

In [ ]:
merlin.undeploy(v)